In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드|
load_dotenv()

True

In [3]:
import os

# LangSmith 추적 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = ""

In [66]:
from langchain_community.document_loaders import JSONLoader

# JSONLoader를 사용하여 데이터 로드
json_loader = JSONLoader(
    file_path="./data-sample.json",
    jq_schema=".",  # 최상위 리스트의 모든 객체를 로드
    text_content=False  # ✅ page_content가 문자열이 아닐 경우 오류 방지
)

docs = json_loader.load()

# 데이터 확인
print(docs)


[Document(metadata={'source': '/home/chae/RAG/data-sample.json', 'seq_num': 1}, page_content="[{'title': '작가사진', 'title_ch': '作家寫眞', 'title_eng': 'Selfportrait', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 1, 'year': '1960', 'size': '41×51', 'materials': '종이에 젤라틴실버프린트', 'category': '사진', 'description': '‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.', 'read_count': 10468}, {'title': '팔괘호', 'title_ch': '八卦壺', 'title_eng': 'Palgwae Vase', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 2, 'year': '1960', 'size': '250×127', 'materials': '캔버스, 종이에 유화 물감', 'category': '회화 II', 'description': '한농(韓農) 한기석(1930-20

In [67]:
docs[0].metadata

{'source': '/home/chae/RAG/data-sample.json', 'seq_num': 1}

In [68]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [69]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [70]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [71]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [72]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 1}

In [73]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

embedding nodes:  50%|█████     | 6/12 [00:00<00:00, 12.33it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Palgwae series', 'Korean artist', 'Color and texture']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Palgwae series', 'Korean artist', 'Color and texture']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Philosophical imagery', 'Bright background', 'Negative space patterns']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Philosophical imagery', 'Bright background', 'Negative space patterns']}
embedding nodes:  67%|██████▋   | 8/12 [00:01<00:00,  5.63it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', 'Green Palgwae and Vase', 'Palgwae Carp', 'Distinctive patterns and colors', 'Korean historical and cultural themes']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', 'Green Palg

In [74]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"How does Han Kisuk, a Korean artist known for ...",[표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는...,Han Kisuk reflects cultural synthesis in his p...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
1,Who is the Korean artist responsible for creat...,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",The Korean artist responsible for creating the...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
2,Who is the Korean artist responsible for creat...,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",The Korean artist responsible for creating the...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
3,Who is the Korean artist known for his work ti...,"[[{'title': '작가사진', 'title_ch': '作家寫眞', 'title...","The Korean artist known for his work titled ""S...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
4,"How does Han Kisuk's artwork, known for blendi...",[표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는...,Han Kisuk's artwork reflects cultural synthesi...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
5,Who is the Korean artist known for creating th...,"['category': '회화 II', 'description': '한농(韓農) 한...",The Korean artist known for creating the artwo...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
6,What is depicted in HAN Kisuk's artwork titled...,"[[{'title': '작가사진', 'title_ch': '作家寫眞', 'title...","HAN Kisuk's artwork titled ""Selfportrait"" from...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
7,What elements are depicted in HAN Kisuk's artw...,"['1970', 'size': '75×61', 'materials': '캔버스, 종...","HAN Kisuk's artwork ""The Moon and Two Cats"" cr...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
8,What are some characteristics of HAN Kisuk's a...,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",HAN Kisuk's artwork from the 1960s is characte...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
9,How do color and texture contribute to the eff...,"['category': '회화 II', 'description': '한농(韓農) 한...","In Han Kisuk's paintings from the 1960s, color...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True


In [75]:
# DataFrame을 CSV 파일로 저장
df.to_csv("dataset_csv/gpt_qa_0213.csv", index=False)

In [76]:
import pandas as pd

df = pd.read_csv('dataset_csv/gpt_qa_0213.csv')

In [77]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [78]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


In [79]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,"How does Han Kisuk, a Korean artist known for ...","[""표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이...",Han Kisuk reflects cultural synthesis in his p...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,"'팔괘잉어', '청팔괘와 화병' 등 역사와 현대의 문화적 요소를 혼합하는 것으로 유...",한기석 작가는 한국의 역사적 기원 속에서 의미 있는 형태를 찾고 과거와 현재의 문화...
1,Who is the Korean artist responsible for creat...,"[""'1970', 'size': '75×61', 'materials': '캔버스, ...",The Korean artist responsible for creating the...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 만든 한국 작가는 누구인가요?,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 제작한 한국 작가는 한기석 작...
2,Who is the Korean artist responsible for creat...,"[""'1970', 'size': '75×61', 'materials': '캔버스, ...",The Korean artist responsible for creating the...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'달과 고양이 두 마리'와 '팔괘'라는 작품을 만든 한국 작가는 누구인가요?,'달과 고양이 두 마리'와 '팔괘'를 제작한 한국 작가는 한기석 작가입니다.
3,Who is the Korean artist known for his work ti...,"[""[{'title': '작가사진', 'title_ch': '作家寫眞', 'titl...","The Korean artist known for his work titled ""S...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 화가는 누구인가요?,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 작가는 한기석 작가입니다.
4,"How does Han Kisuk's artwork, known for blendi...","[""표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이...",Han Kisuk's artwork reflects cultural synthesi...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,한국의 역사적 영향과 현대 문화적 요소를 혼합하는 것으로 유명한 한기석 작가의 작품...,한기석 작가의 작품은 한국의 역사적 기원에서 사상적 뿌리를 찾고 과거와 현재의 문화...


In [80]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"[""표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,"'팔괘잉어', '청팔괘와 화병' 등 역사와 현대의 문화적 요소를 혼합하는 것으로 유...",한기석 작가는 한국의 역사적 기원 속에서 의미 있는 형태를 찾고 과거와 현재의 문화...
1,"[""'1970', 'size': '75×61', 'materials': '캔버스, ...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 만든 한국 작가는 누구인가요?,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 제작한 한국 작가는 한기석 작...
2,"[""'1970', 'size': '75×61', 'materials': '캔버스, ...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'달과 고양이 두 마리'와 '팔괘'라는 작품을 만든 한국 작가는 누구인가요?,'달과 고양이 두 마리'와 '팔괘'를 제작한 한국 작가는 한기석 작가입니다.
3,"[""[{'title': '작가사진', 'title_ch': '作家寫眞', 'titl...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 화가는 누구인가요?,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 작가는 한기석 작가입니다.
4,"[""표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이...",simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,한국의 역사적 영향과 현대 문화적 요소를 혼합하는 것으로 유명한 한기석 작가의 작품...,한기석 작가의 작품은 한국의 역사적 기원에서 사상적 뿌리를 찾고 과거와 현재의 문화...


In [81]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("dataset_csv/gpt_qa_translate_0213.csv", index=False)

### RAGAS를 활용한 평가

In [82]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 10
})

In [83]:
import ast


# contexts 컬럼의 문자열을 리스트로 변환
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}

test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map: 100%|██████████| 10/10 [00:00<00:00, 487.48 examples/s]

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 10
})


In [84]:
test_dataset[1]["contexts"]

["'1970', 'size': '75×61', 'materials': '캔버스, 종이에 유화 물감', 'category': '회화 II', 'description': '한농(韓農) 한기석(1930-2011)의 1960년대에 제작된 <팔괘>는 은은한 녹색의 바탕색 안에서 사각형과 일정한 길이의 가로로 그어진 선과 점들이 세로 그림의 중앙을 메우고 이런 패턴이 양 옆으로 스며들 듯이 그려져 있다.', 'read_count': 125}, {'title': '달과 두 고양이', 'title_ch': 'N/A', 'title_eng': 'The Moon and Two Cats', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 9, 'year': '1960', 'size': '50×60', 'materials': '캔버스에 유화 물감, 왁스', 'category': '회화 II', 'description': '한농(韓農) 한기석(1930-2011)이 즐겨 다루는 주제는 고양이, 잠자리, 산, 달, 감, 여체 등으로 대략 삼십 가지로 제한되어 있는데, 선으로 단순화하여 표현하고 있다.1960년대에 제작된 <달과 두 고양이>작품은 검정색 배경에 노란색의 엎드려 있는 두 마리의 고양이와 달을 간결하게 단순화하여 화면을 구성하고 있으며, 무난한 색이 편안한 느낌을 준다.', 'read_count': 21771}, {'title': '팔괘', 'title_ch': '八卦', 'title_eng': 'Palgwae', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 10, 'year': '1960', 'size': '61×46', 'materials': '캔버스에 유화 물감, 왁스', 'category': '회화 II', 'description': '한농(韓農) 한기석(1930-2011)의 1960년대에 제작된 <팔괘>는 노란색과 연한 주황색의 배경에 같

In [85]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

["'팔괘잉어', '청팔괘와 화병' 등 역사와 현대의 문화적 요소를 혼합하는 것으로 유명한 한국화가 한기석 화백은 어떻게 문화적 융합을 그림에 반영했을까요?",
 "'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 만든 한국 작가는 누구인가요?",
 "'달과 고양이 두 마리'와 '팔괘'라는 작품을 만든 한국 작가는 누구인가요?"]

In [86]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114_docx"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [87]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." in Korean.

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in **Korean**.
Always use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 20,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.8      # 다양성과 관련성 간 균형
    }
)

import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### 모델 결과\n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### 모델 결과\n\n{llm_output.strip()}\n\n"


from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)




Loading checkpoint shards: 100%|██████████| 7/7 [00:29<00:00,  4.23s/it]
Device set to use cuda


In [88]:
answer = chain.batch(batch_dataset)
answer[:3]

['### 모델 결과\n\n한기석 화백은 문화적 융합을 그의 작품에 매우 독창적이고 의미 깊게 반영합니다. 특히 다음과 같은 방식으로 역사와 현대, 그리고 동양과 서양의 문화적 요소를 혼합합니다:\n\n### 1. **전통 문양과 현대적 해석의 결합**\n   - **팔괘 패턴 활용**: 작품들에서 자주 등장하는 팔괘(八卦) 패턴은 동양 철학과 전통 문화의 핵심 요소입니다. 예를 들어, `<팔괘붕어>`나 `<팔괘>`에서 볼 수 있듯이, 이 패턴은 현대적인 표현 기법과 결합되어 새로운 의미를 창출합니다. 이는 과거의 지혜와 현대 예술의 감각을 동시에 전달합니다.\n   - **색상과 질감의 혁신**: 전통적인 색상 팔레트와 현대적인 색채 이론을 혼합하여 독특한 시각적 경험을 제공합니다. `<팔괘붕어>`에서 자주색과 푸른색을 사용한 것처럼, 전통적인 색상 선택에 현대적인 감각을 더해 작품에 생명력을 불어넣습니다.\n\n### 2. **자연과 상징성의 결합**\n   - **자연 모티브의 재해석**: `<팔괘붕어>`, `<팔괘호>` 등의 작품에서 자연 모티브인 붕어와 항아리를 통해 전통적인 상징성을 현대적으로 재해석합니다. 이러한 모티브들은 한국의 자연과 문화적 상징성을 담고 있으며, 이를 간결하고 양식화된 형태로 표현하여 현대인들에게 친숙한 메시지를 전달합니다.\n   - **촉각적 표현**: 작품에서 사용되는 재료와 기법(예: 왁스와 유화 물감의 조합)은 시각적인 경험뿐 아니라 촉각적인 감각까지 자극하여, 관람자에게 다차원적인 예술 체험을 선사합니다.\n\n### 3. **다양한 매체의 융합**\n   - **회화와 판화의 교차**: 한기석 화백은 회화뿐만 아니라 판화 작업에서도 팔괘 패턴과 자연 모티브를 활용합니다. `<팔괘붕어>` (판화)와 같은 작품들은 동일한 주제를 다른 매체를 통해 재해석함으로써, 다양한 표현 방식을 통해 문화적 융합을 보여줍니다.\n\n이러한 방식으로 한기석 화백은 과거와 현재, 동양과 서양의 문화적 요소를 조화롭게 결합하여 독특하고 

In [89]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

### 답변 평가

In [90]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 40/40 [03:14<00:00,  4.86s/it]


{'context_precision': 1.0000, 'faithfulness': 0.1250, 'answer_relevancy': 0.5021, 'context_recall': 0.4667}

In [91]:
result_df = result.to_pandas()
result_df.head()

,question,contexts,answer,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,"'팔괘잉어', '청팔괘와 화병' 등 역사와 현대의 문화적 요소를 혼합하는 것으로 유...",[표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는...,### 모델 결과\n\n한기석 화백은 문화적 융합을 그의 작품에 매우 독창적이고 의...,한기석 작가는 한국의 역사적 기원 속에서 의미 있는 형태를 찾고 과거와 현재의 문화...,1.0,0.75,0.000000,0.0
1,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 만든 한국 작가는 누구인가요?,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",### 모델 결과\n\n두 작품 모두 **한기석 (HAN Kisuk)** 작가에 의...,'달과 고양이 두 마리'와 '팔괘'라는 제목의 작품을 제작한 한국 작가는 한기석 작...,1.0,0.00,0.814963,1.0
2,'달과 고양이 두 마리'와 '팔괘'라는 작품을 만든 한국 작가는 누구인가요?,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",### 모델 결과\n\n한국 작가 **한기석** (HAN Kisuk)이 '달과 고양...,'달과 고양이 두 마리'와 '팔괘'를 제작한 한국 작가는 한기석 작가입니다.,1.0,0.00,0.808320,1.0
3,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 화가는 누구인가요?,"[[{'title': '작가사진', 'title_ch': '作家寫眞', 'title...",### 모델 결과\n\n두 작품 모두로 유명한 한국 화가는 **고희동**입니다. 하...,'자화상'과 '팔괘 꽃병'이라는 작품으로 유명한 한국 작가는 한기석 작가입니다.,1.0,0.00,0.000000,1.0
4,한국의 역사적 영향과 현대 문화적 요소를 혼합하는 것으로 유명한 한기석 작가의 작품...,[표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는...,### 모델 결과\n\n한기석 작가는 한국의 역사적 뿌리와 서양의 현대 문화 사이의...,한기석 작가의 작품은 한국의 역사적 기원에서 사상적 뿌리를 찾고 과거와 현재의 문화...,1.0,0.00,0.000000,0.0


In [92]:
result_df.to_csv("dataset_csv/ragas_result_0213.csv", index=False)

In [93]:
result_df.loc[:, "context_precision":"context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.0,0.750000,0.000000,0.0
1,1.0,0.000000,0.814963,1.0
2,1.0,0.000000,0.808320,1.0
3,1.0,0.000000,0.000000,1.0
4,1.0,0.000000,0.000000,0.0
5,1.0,0.333333,0.843497,1.0
6,1.0,0.166667,0.872183,0.0
7,1.0,0.000000,0.811257,0.2
8,1.0,0.000000,0.870563,0.0
9,1.0,0.000000,0.000000,NaN
